### 1. LLM 链

In [31]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

In [2]:
llm.invoke("how can langsmith help with testing?")

"\nLangsmith is a programming language that can be used for testing purposes. Here are some ways in which Langsmith can help with testing:\n\n1. **Automated Testing**: Langsmith provides a built-in support for automated testing through its `test` module. You can write test cases in Langsmith and run them automatically to ensure that your code works as expected.\n2. **Mocking**: Langsmith's mocking system allows you to easily create mock implementations of external dependencies, making it easier to isolate the parts of your code that are being tested. This can help you write more targeted tests and reduce the risk of introducing bugs during testing.\n3. **Property-based Testing**: Langsmith's property-based testing support allows you to define tests based on mathematical properties of the input data, rather than just checking individual test cases. This can help you cover a wider range of inputs and scenarios, and identify potential issues earlier in the development process.\n4. **Test-

In [3]:
# 使用提示模板来指导大模型的响应，提示模板用于将原始用户输入转换为更好的 LLM 输入

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

# 将它们组合成一个简单的LLM链

chain = prompt | llm

# 现在，我们调用它并提出同样的问题，对于技术作家来说，它应该以更合适的语气做出响应！
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAh, an excellent question! As a world-class technical documentation writer, I must say that Langsmith is an invaluable tool for any testing process. Here are some ways Langsmith can assist in testing:\n\n1. **Automated Content Generation**: With Langsmith's AI-powered content generation capabilities, you can automate the creation of test cases, test scripts, and other documentation. This saves time and effort, allowing your team to focus on more critical tasks.\n2. **Consistency Checking**: Langsmith can help ensure consistency in your testing process by identifying and flagging any inconsistencies in your documentation. This helps maintain a uniform standard across all tests, which is crucial for accurate and reliable test results.\n3. **Error Detection**: Langsmith's advanced AI algorithms can detect errors and discrepancies in your technical documents that might otherwise go unnoticed. This can include things like incorrect syntax, formatting issues, or even logical inconsistenci

In [4]:
# chat模型的输出是一条小溪，然后使用字符串可能更方便些。这里添加简单的输出解析器将chat信息转换成字符串

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

# 将解析器添加之前的链上
chain = prompt | llm | output_parser

# 再次调用
chain.invoke({"input": "how can langsmith help with testing?"})

'\nAs a world-class technical documentation writer, I can provide valuable insights and tools to assist with testing. Here are some ways Langsmith can help:\n\n1. Content Creation: Langsmith can create comprehensive and accurate content for your product or service, including user manuals, technical guides, and troubleshooting resources. This content can be used to train testers on how to use the product effectively, ensuring that they are able to test it thoroughly and identify any issues.\n2. Style Guides: Langsmith can help create style guides for your product or service, which can be used to ensure consistency in testing across different teams and locations. A well-defined style guide can also help testers identify and report on issues more efficiently.\n3. Test Planning: Langsmith can assist with test planning by developing test plans and scenarios that cover all aspects of the product or service. This can help ensure that tests are thorough, well-organized, and effective in identi

### 2. 检索链 

In [32]:
# 1. 加载要索引的数据，使用 WebBaseLoader。该组件依赖于BeautifulSoup(pip install beautifulsoup4)

from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

# 接下来，我们需要将其索引到向量存储中。这需要一些组件，即嵌入模型和向量存储。
# 对于嵌入模型，我们再次运行本地模型访问的示例。

from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

# 现在，我们可以使用此嵌入模型将文档提取到向量存储中。为了简单起见，我们将使用一个简单的本地向量库FAISS(pip install faiss-cpu)
# 接下来建立索引

from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

# 现在已经在矢量存储中索引了这些数据，这里创建一个链
# 该链将接受传入的问题，查找相关的文档，然后将这些文档于原始问题一起传递给LLM并要求回答原始问题。

from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template(
"""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""
)

document_chain = create_stuff_documents_chain(llm, prompt)

# 运行链

from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'\nBased on the provided context, Langsmith can help with testing by allowing users to visualize their test results.'

In [33]:
# 但是，我们希望文档首先来自刚刚设置的检索器
# 这样，对于给定的问题，可以使用检索器动态选择最相关的文档并将其传递进去。

from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
# 基于输入，先检索相关文档作为context，然后将相关内容传递给后面链处理
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# 在可以调用这个链。这将返回一个字典 - 来自LLM的响应位于answer键中
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])


According to the provided context, LangSmith can help with testing in several ways:

1. Creating test cases: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications.
2. Debugging: LangSmith provides native rendering of chat messages, functions, and retrieve documents, making it easy to identify and debug issues in the application.
3. Comparing test runs: LangSmith's comparison view allows developers to track and diagnose regressions in test scores across multiple revisions of their application.
4. Beta testing: LangSmith enables developers to collect more data on how their LLM applications are performing in real-world scenarios, helping them understand how the app is performing and identify areas for improvement.
5. Annotating traces: LangSmith supports sending runs to annotation queues, allowing annotators to closely inspect interesting traces and annotate them with respect to differen

### 3. 对话检索

In [9]:
# 更新检索
# 为了更新检索，我们将创建一个新链。该链将接受最近的输入 ( input) 和对话历史记录 ( chat_history) 并使用 LLM 生成搜索查询

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user","{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

# 通过传入用户提出后续问题的实例来测试这一点
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubProxyCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new LLM appli

### 4. 代理

由于需要OpenAI的API-KEY，所以这里跳过

In [18]:
# from langchain import hub
# from langchain.agents import AgentExecutor, create_react_agent
# from langchain_community.tools.tavily_search import TavilySearchResults
# from langchain_community.llms import Ollama
# import os


# os.environ["TAVILY_API_KEY"] = "xxxxxxx"

# llm = Ollama(model="llama2")

# tools = [TavilySearchResults(max_results=5)]

# # Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/react")


# # Construct the ReAct agent
# agent = create_react_agent(llm, tools, prompt)

# # Create an agent executor by passing in the agent and tools
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# agent_executor.invoke({"input": "what is LangChain?"})

In [19]:
# from langchain_community.tools.tavily_search import TavilySearchResults

# tool = TavilySearchResults()
# tool.invoke({"query": "What is the weather like in Shanghai today?"})

In [29]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.retrievers import TavilySearchAPIRetriever

# os.environ["TAVILY_API_KEY"] = "xxxxxxx"

retriever = TavilySearchAPIRetriever(k=3)

prompt = ChatPromptTemplate.from_template(
    """Answer the question based on the context provided.

Context: {context}

Question: {question}"""
)
chain = (
    RunnablePassthrough.assign(context=(lambda x: x["question"]) | retriever)
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "帮我写个python示例代码，读取本地一个文件的内容"})

' certainly! Here is an example of how to read the contents of a local file in Python:\n```\n# Open the file using the open() function\nwith open("file.txt", "r") as f:\n    # Read the contents of the file\n    contents = f.read()\n\n# Print the contents of the file\nprint(contents)\n```\nIn this example, we use the `open()` function to open a file named `"file.txt"` in read mode (`"r"`). The `with` statement ensures that the file is properly closed after use, regardless of whether an exception is thrown or not. The `read()` method of the file object returns the entire contents of the file, which we then print to the console.\n\nNote that you will need to replace `"file.txt"` with the actual name and location of the file you want to read.'

In [27]:
chain.invoke({"question": "How is the weather in shanghai on March 27, 2024?"})

'Based on the information provided by the documents, the weather in Shanghai on March 27, 2024 is expected to be around 27.8 °C (82.1 °F) during the day and 18.9 °C (66 °F) at night.'

In [28]:
chain.invoke({"question": "How is the weather in shanghai today?"})

'Based on the context provided, the current weather in Shanghai today is sunny with a temperature of 73°F (23°C). This information can be found on the following pages:\n\n* Page 1: Hour-by-Hour Forecast for Shanghai - <https://www.timeanddate.com/weather/china/shanghai/hourly>\n* Page 2: Current weather in Shanghai and forecast for today, tomorrow, and next 14 days - <https://www.timeanddate.com/weather/china/shanghai>\n* Page 3: Weather Today for Shanghai, Shanghai, China - <https://www.accuweather.com/en/cn/shanghai/106577/weather-today/>\n\nAll of these sources indicate that the weather in Shanghai today is sunny and mild, with a high temperature of 23°C (73°F) and low humidity.'